In [4]:
sorted(['a1', 'a001', 'a002', ])

['a001', 'a002', 'a1']

In [6]:
N_JP0 = 4384  # Japanese population size (inital)
r_JP = 0.0129  # Japanese growth rate
generation_time = 29
T_CH_JP = int(9000 / generation_time)  # time of Han/Japanese split

N_JP0 * math.exp(r_JP * T_CH_JP)

239119.05109482087

In [7]:
import sys
import math
import tskit
import tszip
import msprime
import stdpopsim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import collections
import multiprocessing
import itertools
from contextlib import redirect_stdout
import subprocess

In [8]:
msprime.__version__

'1.1.0'

In [2]:
tskit.__version__, msprime.__version__, stdpopsim.__version__

('0.4.1', '1.1.0', '0.1.2')

In [3]:
_species = stdpopsim.get_species("HomSap")
_yri_population = stdpopsim.Population(
    id="YRI", description="1000 Genomes YRI (Yoruba)"
)
_ceu_population = stdpopsim.Population(
    id="CEU",
    description=(
        "1000 Genomes CEU (Utah Residents (CEPH) with Northern and "
        "Western European Ancestry"
    ),
)
_chb_population = stdpopsim.Population(
    id="CHB", description="1000 Genomes CHB (Han Chinese in Beijing, China)"
)

def _ooa_4pop():
    id = "OutOfAfrica_4J17"
    description = "4 population out of Africa"
    long_description = """
        Demographic model for a four population out-of-Africa history,
        taken from Jouganous et al. (2017). Parameter values were taken
        from table 4 in the main text. This model was fit based on joint
        allele frequecy spectrum (AFS) data from 1000 Genomes exomes
        from the YRI, CEU, CHB, and JPT poulation samples. The demography
        follows the previous three-populations out-of-Africa models
        with an additional population split in Asia leading to the
        Japanese (JPT) population. Parameter values were estimated with
        the program Moments assuming a mutation rate of 1.44e-8 and a
        generation time of 29 years.
    """
    populations = [
        _yri_population,
        _ceu_population,
        _chb_population,
        stdpopsim.Population(
            id="JPT",
            description="1000 Genomes JPT (Japanese in Tokyo, Japan)",
            sampling_time=0,
        ),
    ]
    citations = [
        stdpopsim.Citation(
            author="Jouganous et al.",
            year="2017",
            doi="https://doi.org/10.1534/genetics.117.200493",
            reasons={stdpopsim.CiteReason.DEM_MODEL},
        )
    ]

    generation_time = 29
    mutation_rate = 1.44e-8  # from Gravel et al. 2013 (Plos Gen)

    # Parameter values from Table 4 (in bold)

    # Population sizes
    N_A = 11293  # ancestral YRI population size
    N_AF = 23721  # YRI population size
    N_B = 2831  # B population size (out-of-Africa)
    N_EU0 = 2512  # CEU population size (intial)
    N_AS0 = 1019  # Asian population size (inital)
    N_JP0 = 4384  # Japanese population size (inital)

    # growth rates
    r_EU = 0.0016  # European growth rate
    r_JP = 0.0129  # Japanese growth rate
    r_AS = 0.0026  # Asian growth rate

    # migration rates
    m_AF_B = 16.8e-5  # YRI <--> B (out-of-Africa)
    m_AF_EU = 1.14e-5  # YRI <--> CEU
    m_AF_AS = 0.56e-5  # YRI <--> CHB
    m_EU_AS = 4.75e-5  # European <--> CHB
    m_CH_JP = 3.3e-5  # CHBHan <--> JPT

    # epochs (specified in years, converted to generations)
    T_AF = int(357000 / generation_time)  # time ancestral to African
    T_B = int(119000 / generation_time)  # time of Out of Africa split
    T_EU_AS = int(46000 / generation_time)  # time of Europe/Asia split
    T_CH_JP = int(9000 / generation_time)  # time of Han/Japanese split

    # also
    r_AF = 0  # YRI and B growth rate

    # A and YRI
    pop0 = msprime.PopulationConfiguration(
        initial_size=N_AF * math.exp(r_AF * T_AF),
        growth_rate=r_AF,
        metadata=populations[0].asdict(),
    )

    # B and CEU
    pop1 = msprime.PopulationConfiguration(
        initial_size=N_EU0 * math.exp(r_EU * T_EU_AS),
        growth_rate=r_EU,
        metadata=populations[1].asdict(),
    )

    # Asian and CHB
    pop2 = msprime.PopulationConfiguration(
        initial_size=N_AS0 * math.exp(r_AS * T_EU_AS),
        growth_rate=r_AS,
        metadata=populations[2].asdict(),
    )

    # JPT
    pop3 = msprime.PopulationConfiguration(
        initial_size=N_JP0 * math.exp(r_JP * T_CH_JP),
        growth_rate=r_JP,
        metadata=populations[3].asdict(),
    )

    population_configurations = [pop0, pop1, pop2, pop3]

    migration_matrix = [
        [0, m_AF_EU, m_AF_AS, 0],
        [m_AF_EU, 0, m_EU_AS, 0],
        [m_AF_AS, m_EU_AS, 0, m_CH_JP],
        [0, 0, m_CH_JP, 0],
    ]

    # Demographic events from most recent -> oldest

    # JPT splits from CHB
    JPT_event = [
        # seg migration rate JPT <--> CHB to zero
        msprime.MigrationRateChange(time=T_CH_JP, rate=0.0, matrix_index=(2, 3)),
        msprime.MigrationRateChange(time=T_CH_JP, rate=0.0, matrix_index=(3, 2)),
        # JPT splits from CHB
        msprime.MassMigration(time=T_CH_JP, source=3, destination=2, proportion=1.0),
    ]

    # CEU and CHB merge into B
    AS_event = [
        # set migration rates to zero
        msprime.MigrationRateChange(time=T_EU_AS, rate=0.0),
        # CHB splits from CEU
        msprime.MassMigration(time=T_EU_AS, source=2, destination=1, proportion=1.0),
        # change size of B to N_B
        # set growth rates in B to zero
        msprime.PopulationParametersChange(
            time=T_EU_AS, initial_size=N_B, growth_rate=0.0, population_id=1
        ),
        # set up migration between YRI and B
        msprime.MigrationRateChange(time=T_EU_AS, rate=m_AF_B, matrix_index=(0, 1)),
        msprime.MigrationRateChange(time=T_EU_AS, rate=m_AF_B, matrix_index=(1, 0)),
    ]

    # Out-of-Africa event
    EU_event = [
        # B splits from YRI
        msprime.MigrationRateChange(time=T_B, rate=0.0),
        msprime.MassMigration(time=T_B, source=1, destination=0, proportion=1.0),
    ]

    # Change size of YRI/African population
    AF_event = [
        msprime.PopulationParametersChange(time=T_AF, initial_size=N_A, population_id=0)
    ]

    demographic_events = JPT_event + AS_event + EU_event + AF_event

    return stdpopsim.DemographicModel(
        id=id,
        description=description,
        long_description=long_description,
        populations=populations,
        citations=citations,
        generation_time=generation_time,
        #mutation_rate=mutation_rate,
        population_configurations=population_configurations,
        migration_matrix=migration_matrix,
        demographic_events=demographic_events,
    )


_species.add_demographic_model(_ooa_4pop())

In [4]:
_species.get_demographic_model("OutOfAfrica_4J17").debug()

DemographyDebugger
╠════════════════════════════════╗
║ Epoch[0]: [0, 310) generations ║
╠════════════════════════════════╝
╟    Populations (total=4 active=4)
║    ┌───────────────────────────────────────────────────────────────────────────────────────┐
║    │       │      start│       end│growth_rate  │  pop_0   │  pop_1   │  pop_2   │  pop_3  │
║    ├───────────────────────────────────────────────────────────────────────────────────────┤
║    │  pop_0│    23721.0│   23721.0│ 0           │    0     │ 1.14e-05 │ 5.6e-06  │    0    │
║    │  pop_1│    31775.0│   19349.7│ 0.0016      │ 1.14e-05 │    0     │ 4.75e-05 │    0    │
║    │  pop_2│    62955.1│   28118.3│ 0.0026      │ 5.6e-06  │ 4.75e-05 │    0     │ 3.3e-05 │
║    │  pop_3│   239119.1│    4384.0│ 0.0129      │    0     │    0     │ 3.3e-05  │    0    │
║    └───────────────────────────────────────────────────────────────────────────────────────┘
╟    Events @ generation 310
║    ┌─────────────────────────────────────────────

In [5]:
def _ooa_4popADMIXTURE():
    id = "OutOfAfrica_4J17_ADMIXTURE"
    description = "4 population out of Africa with admixture"
    long_description = """
        Demographic model for a four population out-of-Africa history,
        taken from Jouganous et al. (2017), 
        with admixture and recent population growth added.  
    """
    populations = [
        _yri_population,
        _ceu_population,
        _chb_population,
        stdpopsim.Population(
            id="JPT",
            description="1000 Genomes JPT (Japanese in Tokyo, Japan)",
            sampling_time=0,
        ),
        stdpopsim.Population(
            id="ADMIX",
            description="Admixed population",
            sampling_time=0,
        ),        
    ]
    citations = [
        stdpopsim.Citation(
            author="Jouganous et al.",
            year="2017",
            doi="https://doi.org/10.1534/genetics.117.200493",
            reasons={stdpopsim.CiteReason.DEM_MODEL},
        )
    ]

    generation_time = 29
    # Parameter values from Table 4 (in bold)

    # Population sizes
    N_A = 11293  # ancestral YRI population size
    N_AF = 23721  # YRI population size
    N_B = 2831  # B population size (out-of-Africa)
    N_EU0 = 2512  # CEU population size (intial)
    N_AS0 = 1019  # Asian population size (inital)
    N_JP0 = 4384  # Japanese population size (inital)

    # growth rates
    r_EU = 0.0016  # European growth rate
    r_JP = 0.0129  # Japanese growth rate
    r_AS = 0.0026  # Asian growth rate
    
    # added recent growth
    recent_AF = 0.1438809454231192
    recent_EU = 0.11464909373821389
    recent_AS = 0.04627484317475039

    # migration rates
    m_AF_B = 16.8e-5  # YRI <--> B (out-of-Africa)
    m_AF_EU = 1.14e-5  # YRI <--> CEU
    m_AF_AS = 0.56e-5  # YRI <--> CHB
    m_EU_AS = 4.75e-5  # European <--> CHB
    m_CH_JP = 3.3e-5  # CHBHan <--> JPT

    # epochs (specified in years, converted to generations)
    T_AF = int(357000 / generation_time)  # time ancestral to African
    T_B = int(119000 / generation_time)  # time of Out of Africa split
    T_EU_AS = int(46000 / generation_time)  # time of Europe/Asia split
    T_CH_JP = int(9000 / generation_time)  # time of Han/Japanese split

    # also
    r_AF = 0  # YRI and B growth rate

    # A and YRI
    pop0 = msprime.PopulationConfiguration(
        #initial_size=N_AF * math.exp(r_AF * T_AF),
        #growth_rate=r_AF,
        initial_size=100000,
        growth_rate=recent_AF,
        metadata=populations[0].asdict(),
    )

    # B and CEU
    pop1 = msprime.PopulationConfiguration(
        #initial_size=N_EU0 * math.exp(r_EU * T_EU_AS),
        #growth_rate=r_EU,
        initial_size=100000,
        growth_rate=recent_AF,
        metadata=populations[1].asdict(),
    )

    # Asian and CHB
    pop2 = msprime.PopulationConfiguration(
        #initial_size=N_AS0 * math.exp(r_AS * T_EU_AS),
        #growth_rate=r_AS,
        initial_size=100000,
        growth_rate=recent_AS,
        metadata=populations[2].asdict(),
    )

    # JPT
    pop3 = msprime.PopulationConfiguration(
        initial_size=N_JP0 * math.exp(r_JP * T_CH_JP),
        growth_rate=r_JP,
        metadata=populations[3].asdict(),
    )
    
    # ADMIX
    pop4 = msprime.PopulationConfiguration(
        initial_size=50000 * math.exp(0.05 * 12),
        growth_rate=0.05,
        metadata=populations[4].asdict(),
    )

    population_configurations = [pop0, pop1, pop2, pop3, pop4]

    migration_matrix = [
        [0, m_AF_EU, m_AF_AS, 0, 0],
        [m_AF_EU, 0, m_EU_AS, 0, 0],
        [m_AF_AS, m_EU_AS, 0, m_CH_JP, 0],
        [0, 0, m_CH_JP, 0, 0],
        [0,0,0,0,0]
    ]

    # Demographic events from most recent -> oldest
    
    # growth
    stop_recent_growth = [
        msprime.PopulationParametersChange(
            population_id=0, time=10, growth_rate=r_AF
        ),
        msprime.PopulationParametersChange(
            population_id=1, time=10, growth_rate=r_EU
        ),
        msprime.PopulationParametersChange(
            population_id=2, time=10, growth_rate=r_AS
        ),
        
    ]
    
    # admixture
    admixture = [
        msprime.MassMigration(time=12, source=4, destination=0, proportion=.15),
        msprime.MassMigration(time=12, source=4, destination=1, proportion=.15),
        msprime.MassMigration(time=12, source=4, destination=2, proportion=.30),
        msprime.MassMigration(time=12, source=4, destination=3, proportion=.30),
    ]
    

    # JPT splits from CHB
    JPT_event = [
        # seg migration rate JPT <--> CHB to zero
        msprime.MigrationRateChange(time=T_CH_JP, rate=0.0, matrix_index=(2, 3)),
        msprime.MigrationRateChange(time=T_CH_JP, rate=0.0, matrix_index=(3, 2)),
        # JPT splits from CHB
        msprime.MassMigration(time=T_CH_JP, source=3, destination=2, proportion=1.0),
    ]

    # CEU and CHB merge into B
    AS_event = [
        # set migration rates to zero
        msprime.MigrationRateChange(time=T_EU_AS, rate=0.0),
        # CHB splits from CEU
        msprime.MassMigration(time=T_EU_AS, source=2, destination=1, proportion=1.0),
        # change size of B to N_B
        # set growth rates in B to zero
        msprime.PopulationParametersChange(
            time=T_EU_AS, initial_size=N_B, growth_rate=0.0, population_id=1
        ),
        # set up migration between YRI and B
        msprime.MigrationRateChange(time=T_EU_AS, rate=m_AF_B, matrix_index=(0, 1)),
        msprime.MigrationRateChange(time=T_EU_AS, rate=m_AF_B, matrix_index=(1, 0)),
    ]

    # Out-of-Africa event
    EU_event = [
        # B splits from YRI
        msprime.MigrationRateChange(time=T_B, rate=0.0),
        msprime.MassMigration(time=T_B, source=1, destination=0, proportion=1.0),
    ]

    # Change size of YRI/African population
    AF_event = [
        msprime.PopulationParametersChange(time=T_AF, initial_size=N_A, population_id=0)
    ]

    demographic_events = stop_recent_growth + admixture + JPT_event + AS_event + EU_event + AF_event

    return stdpopsim.DemographicModel(
        id=id,
        description=description,
        long_description=long_description,
        populations=populations,
        citations=citations,
        generation_time=generation_time,
        #mutation_rate=mutation_rate,
        population_configurations=population_configurations,
        migration_matrix=migration_matrix,
        demographic_events=demographic_events,
    )


_species.add_demographic_model(_ooa_4popADMIXTURE())

In [6]:
_species.get_demographic_model("OutOfAfrica_4J17_ADMIXTURE").debug()

DemographyDebugger
╠═══════════════════════════════╗
║ Epoch[0]: [0, 10) generations ║
╠═══════════════════════════════╝
╟    Populations (total=5 active=5)
║    ┌────────────────────────────────────────────────────────────────────────────────────────────────┐
║    │       │      start│        end│growth_rate  │  pop_0   │  pop_1   │  pop_2   │  pop_3  │ pop_4 │
║    ├────────────────────────────────────────────────────────────────────────────────────────────────┤
║    │  pop_0│   100000.0│    23721.0│ 0.144       │    0     │ 1.14e-05 │ 5.6e-06  │    0    │   0   │
║    │  pop_1│   100000.0│    23721.0│ 0.144       │ 1.14e-05 │    0     │ 4.75e-05 │    0    │   0   │
║    │  pop_2│   100000.0│    62955.1│ 0.0463      │ 5.6e-06  │ 4.75e-05 │    0     │ 3.3e-05 │   0   │
║    │  pop_3│   239119.1│   210179.4│ 0.0129      │    0     │    0     │ 3.3e-05  │    0    │   0   │
║    │  pop_4│    91105.9│    55258.5│ 0.05        │    0     │    0     │    0     │    0    │   0   │
║    └─────

In [8]:
species_name = "HomSap"
model_name = "OutOfAfrica_4J17_ADMIXTURE"
chromosome = "chr22"

remember_gen = 20
ancestral_Ne = 11293
mut_rate = 1.44e-8

length_multiplier=.1

In [9]:
species = stdpopsim.get_species(species_name)
model = species.get_demographic_model(model_name)
contig = species.get_contig(chromosome, length_multiplier=length_multiplier)  # default is a flat genetic map
newmap = msprime.RecombinationMap(
    positions=contig.recombination_map.get_positions(),
    rates= [x*3 for x in contig.recombination_map.get_rates()]
)

newcontig = stdpopsim.Contig(recombination_map = newmap)

In [10]:
WORK_DIR = '/home/kele/Documents/lai/test/test_4pop'

In [11]:
os.system(f'mkdir {WORK_DIR}')
SLiM_script_path = os.path.join(WORK_DIR, 'run.slim')
SLiM_ts_path = os.path.join(WORK_DIR, 'from_slim.ts')
print(SLiM_script_path, SLiM_ts_path)

/home/kele/Documents/lai/test/test_4pop/run.slim /home/kele/Documents/lai/test/test_4pop/from_slim.ts


mkdir: cannot create directory ‘/home/kele/Documents/lai/test/test_4pop’: File exists


In [12]:
for i in range(model.num_populations):
    print(model.population_configurations[i].initial_size)

100000
100000
100000
239119.05109482087
91105.94001952546


In [14]:
samples = model.get_samples(100000,100000,100000,20000)

In [17]:
engine = stdpopsim.get_engine("slim")
with open(SLiM_script_path, "w") as f:
    with redirect_stdout(f):
        _ = engine.simulate(
            model,
            newcontig,
            samples=samples,
            slim_script=True,
            verbosity=2
        )

# add gene conversion
os.system(f"sed -i 's+(recombination_rates, recombination_ends);+(recombination_rates, recombination_ends);\\n    initializeGeneConversion(0.6666666, 300, 1.0);+g' {SLiM_script_path}")


# check the edits
os.system(f'grep {SLiM_ts_path} {SLiM_script_path}')
os.system(f"grep 'initializeGeneConversion' {SLiM_script_path}")


# The goal is to mark all inds 20 generations prior to the present.
# Then use these marked individuals to define the "local ancestry".
# G_end is a variable define in the SLiM script that
# should give the end of the simulation (in generations)


remember_cmd = r's+// Admixture pulses\.+sim.registerLateEvent("s000000", "{dbg(self.source); sim.treeSeqRememberIndividuals(sim.subpopulations.individuals);}", G_end-20, G_end-20);\n    // Admixture pulses\.+g '
remember_cmd


os.system(f"sed -i '{remember_cmd}' {SLiM_script_path}")


# check the edit
os.system(f"grep 's000000' {SLiM_script_path}")

import fileinput
for line in fileinput.input(f'{SLiM_script_path}', inplace=True):
    line = line.rstrip('\r\n')
    if line.strip().startswith('''defineConstant("trees_file",'''):
        print(f'    //defineConstant("trees_file", "{os.path.join(WORK_DIR, "from_slim.ts")});')
    else:
        print(line)
fileinput.close()


    initializeGeneConversion(0.6666666, 300, 1.0);
    sim.registerLateEvent("s000000", "{dbg(self.source); sim.treeSeqRememberIndividuals(sim.subpopulations.individuals);}", G_end-20, G_end-20);


In [ ]:
# Parameters

run_slim = False
run_la = False


npop = 4
admixed_pop_idx = 3
nref = 150000

SEED = 321

WORK_DIR = f'/home/users/waplesr/lai/large/AmericanAdmixture_4B11_100K/{SEED}'
BCFTOOLS = '/home/users/waplesr/lai/large/programs/hts/bcftools-1.13/bcftools'


# ## Species and demographic model

species = stdpopsim.get_species(demo_species)
model = species.get_demographic_model(demo_model)


# ## Genome specification
# We will use **contig** when simulating in SLiM, this simulation is with gene conversion,
# **newcontig** is for use in msprime, and will not include gene conversion.

contig = species.get_contig(chromosome, length_multiplier=length_multiplier)  # default is a flat genetic map

# make a new contig
# constant recombination rate, but 3 times higher rec rate than normal
# this is to allow 2/3rds of rec events to be gene conversions.
newmap = msprime.RecombinationMap(
    positions=contig.recombination_map.get_positions(),
    rates= [x*3 for x in contig.recombination_map.get_rates()]
)

newcontig = stdpopsim.Contig(recombination_map = newmap)

os.system(f'mkdir {WORK_DIR}')
SLiM_script_path = os.path.join(WORK_DIR, 'run.slim')
SLiM_ts_path = os.path.join(WORK_DIR, 'from_slim.ts')
print(SLiM_script_path, SLiM_ts_path)


model.population_configurations[0].initial_size,model.population_configurations[1].initial_size,model.population_configurations[2].initial_size

def get_growth_rate(initial_size, final_size, generations):
    return np.log(final_size/initial_size)/generations

pop0_recent_growth = get_growth_rate(model.population_configurations[0].initial_size, 100000, 10)
pop1_recent_growth = get_growth_rate(model.population_configurations[1].initial_size, 100000, 10)
pop2_recent_growth = get_growth_rate(model.population_configurations[2].initial_size, 100000, 10)

model.population_configurations[0].initial_size = 100000
model.population_configurations[1].initial_size = 100000
model.population_configurations[2].initial_size = 100000

recent_growth = [
 msprime.PopulationParametersChange(population=0, time=0, growth_rate=pop0_recent_growth),
 msprime.PopulationParametersChange(population=1, time=0, growth_rate=pop1_recent_growth),
 msprime.PopulationParametersChange(population=2, time=0, growth_rate=pop2_recent_growth),
]

recent_growth = recent_growth + [
 msprime.PopulationParametersChange(population=0, time=10, growth_rate=0),
 msprime.PopulationParametersChange(population=1, time=10, growth_rate=0.0038),
 msprime.PopulationParametersChange(population=2, time=10, growth_rate=0.0048),
]

model.demographic_events = recent_growth + model.demographic_events


samples = model.get_samples(100000,100000,100000,20000)


# ## Samples

remake_script = False

if remake_script:
    engine = stdpopsim.get_engine("slim")
    with open(SLiM_script_path, "w") as f:
        with redirect_stdout(f):
            _ = engine.simulate(
                model,
                newcontig,
                samples=samples,
                seed=SEED, # not sure if the seed is recognised here
                slim_script=True,
                verbosity=2
            )


            ## Editing the SLiM script prior to execution

            # specify the output file and set up gene conversion

            # change the ts output location
            import fileinput
            for line in fileinput.input(f'{SLiM_script_path}', inplace=True):
                line = line.rstrip('\r\n')
                if line.startswith('''defineConstant("trees_file",'''):
                    print(f'defineConstant("trees_file", "{os.path.join(WORK_DIR, "from_slim.ts")});')
                else:
                    print(line)
            fileinput.close()

            #sed_cmd = f's+"/scratch/*.*.lindstroem.q/tmp*.ts+"{SLiM_ts_path}+g'
            #os.system(f'sed -i {sed_cmd} {SLiM_script_path}')

            # os.system(f"sed -i 's@*scratch*@defineConstant("trees_file", "/home/users/waplesr/lai/large/AmericanAdmixture_4B11_100K/1234567890/from_slim.ts")@g'   {SLiM_script_path}")

            # add gene conversion
            os.system(f"sed -i 's+(recombination_rates, recombination_ends);+(recombination_rates, recombination_ends);\\n    initializeGeneConversion(0.6666666, 300, 1.0);+g' {SLiM_script_path}")


            # check the edits
            os.system(f'grep {SLiM_ts_path} {SLiM_script_path}')
            os.system(f"grep 'initializeGeneConversion' {SLiM_script_path}")


            # The goal is to mark all inds 20 generations prior to the present.
            # Then use these marked individuals to define the "local ancestry".
            # G_end is a variable define in the SLiM script that
            # should give the end of the simulation (in generations)


            remember_cmd = r's+// Admixture pulses\.+sim.registerLateEvent("s000000", "{dbg(self.source); sim.treeSeqRememberIndividuals(sim.subpopulations.individuals);}", G_end-20, G_end-20);\n    // Admixture pulses\.+g '
            remember_cmd


            os.system(f"sed -i '{remember_cmd}' {SLiM_script_path}")


            # check the edit
            os.system(f"grep 's000000' {SLiM_script_path}")


if run_slim:
    os.system(f'/home/users/waplesr/lai/large/programs/slim/build/slim -seed {SEED} {SLiM_script_path} > {SLiM_script_path}.log')

# ## Load in the ts from SLiM


ts = tskit.load(SLiM_ts_path)
ts


# ## Recapitate

# #### Use the original recombination rates for the recapitation, rather than the elevated rates use for gene conversion
# #### Notice this is also a discrete genetic map.

recapmap = msprime.RecombinationMap(
    positions=[0.0, ts.get_sequence_length()],
    rates= contig.recombination_map.get_rates(),
    num_loci = int(ts.get_sequence_length())
)


# 7310 is the ancestral population size in Africa. All the action should be occuring in Africa here.
#
# Recapitation uses the Hudson coalescent.

coalesced_ts = msprime.simulate(
    Ne=ancestral_Ne,
    from_ts=ts,
    random_seed=SEED,
    recombination_map=recapmap,
    population_configurations = [
        msprime.PopulationConfiguration(),
        msprime.PopulationConfiguration(),
        msprime.PopulationConfiguration(),
        msprime.PopulationConfiguration()
    ],
    migration_matrix=None
    )

del ts


for x in range(4):
    y = coalesced_ts.tables.nodes.time[np.where(coalesced_ts.tables.nodes.population==x)[0]]
    plt.plot(np.log10(np.sort(y)+1), label = x)
plt.legend(title = 'population')
plt.ylabel('log10(node age+1)')
plt.xlabel('nodes within each population')
plt.show()


# ## Here I need to edit the ts so that only nodes sampled in the present are marked as 'samples'

tables = coalesced_ts.tables
newnodes = tables.nodes.copy()

flags = tables.nodes.flags
assert flags.sum() == coalesced_ts.num_samples

flags[tables.nodes.time == remember_gen] = 0
assert flags.sum() == np.intersect1d(
                    coalesced_ts.samples(),
                    np.where(tables.nodes.asdict()['time']==0)[0]
                ).size

newnodes.set_columns(
    time = newnodes.time,
    flags=flags,
    population=newnodes.population,
    individual=newnodes.individual,
    metadata=newnodes.metadata,
    metadata_offset=newnodes.metadata_offset,
)




tables.nodes.clear()


for row in newnodes:
    tables.nodes.add_row(flags=row.flags,
                         time=row.time,
                         population=row.population,
                         individual=row.individual,
                         metadata=row.metadata)


coalesced_ts = tables.tree_sequence()
del newnodes
del tables
del flags

for x in range(4):
    print (int(len(coalesced_ts.samples(x))/2))


# downsample the coalesced ts


for pop, nsamp in enumerate([100000,100000,100000,20000]):
    print(pop, nsamp)
    coalesced_ts.samples(pop)[:nsamp]



# ## Add mutations
# Also remove sites with a minor allele count (MAC) of 1. Notice this is removing sites that have MAC=1 at the population level, so a subset of these individuals may still have singletons.



mut_ts = msprime.mutate(
    tree_sequence=coalesced_ts,
    rate=mut_rate,
    random_seed=SEED,
    model = msprime.InfiniteSites(alphabet=msprime.NUCLEOTIDES))
mut_ts


tszip.compress(mut_ts, os.path.join(WORK_DIR, 'mut_ts.tsz'))

mut_ts.dump(os.path.join(WORK_DIR, 'mut_ts.ts'))


# # Filter sites

# # Validation and model checking


site_fst_of_pops = dict()
for x in range(4):
    for y in range(x,4):
        site_fst_of_pops[(x,y)] = mut_ts.Fst(sample_sets = ([mut_ts.samples(population=x), mut_ts.samples(population=y)]))

print(site_fst_of_pops)

for pop in range(4):
    print (int((len(mut_ts.samples(population=pop))/2)))


# # Filtering sites

mut_ts = tszip.decompress(os.path.join(WORK_DIR, 'mut_ts.tsz'))
mac1_ts = strip_singletons(mut_ts)
mac1_ts = strip_adjacent_sites(mac1_ts)
del mut_ts


# # get local ancestry tracts
# needs to work on the entire ts

if run_la:
    #local_df = get_local_ancestry_pop_multi(
    local_df = get_local_ancestry_pop(
        ts=mac1_ts,
        pop=3,
        admixture_time=remember_gen,
        max_samples=20000,
        per_rep=12,
        #n_cores=10
    )

    local_df.to_hdf(os.path.join(WORK_DIR, 'la_df.h5'), key='la_df', complib='blosc:lz4', complevel=9)



local_df = pd.read_hdf(os.path.join(WORK_DIR, 'la_df.h5'), key='la_df')
simp_ts, mapping = mac1_ts.simplify(
    samples=local_df['child'].unique().tolist(),
    map_nodes=True,
    filter_populations=False
)



# remove singletons from the target samples
simp_ts = strip_singletons(simp_ts)


if run_la:
    la_mat, samps = get_la_mat_large(
        ts=simp_ts,
        df=local_df,
        mapping=mapping
    )
    np.savez_compressed(os.path.join(WORK_DIR, 'la_mat.npz'), la_mat=la_mat)

la_mat = np.load(os.path.join(WORK_DIR, 'la_mat.npz'))['la_mat']
plt.plot(la_mat[:,1])


plt.subplots(figsize = (16,16))
sns.heatmap(la_mat[::100].T)


# # Export the data to vcf


keep = np.concatenate(
    [mac1_ts.samples(pop)[:nsamp] for pop, nsamp in enumerate([100000,100000,100000,20000])]
)

downsampled_ts = mac1_ts.simplify(keep)
downsampled_ts


keep_positions = set([s.position for s in simp_ts.sites()])

remove_positions = [i for (i,s) in enumerate(downsampled_ts.sites()) if s.position not in keep_positions]

export_ts = downsampled_ts.delete_sites(remove_positions)
assert export_ts.num_sites == la_mat.shape[0]


# update the individuals table
tables = export_ts.tables
tables.individuals.clear()
tables.nodes.individual = np.repeat(np.array([-1], dtype = 'int32'), len(tables.nodes))
export_ts = tables.tree_sequence()


pop_of_sample = dict(zip(range(len(export_ts.tables.nodes)), export_ts.tables.nodes.population))
pop_of_sample

nind_in_pop = collections.defaultdict(int)
pops = [pop_of_sample[i] for i in export_ts.samples()]
for p in pops:
    nind_in_pop[p] +=1
for p in nind_in_pop:
    nind_in_pop[p] = int(nind_in_pop[p]/2)
ind_labels = []
for p in nind_in_pop:
    for i in range(1, nind_in_pop[p]+1):
        ind_labels.append(f'pop_{p}-ind_{i:05}')
len(ind_labels)


# # Write bcf file

tszip.compress(export_ts, os.path.join(WORK_DIR, 'export_ts.tsz'))

export_ts = tszip.decompress(os.path.join(WORK_DIR, 'export_ts.tsz'))

read_fd, write_fd = os.pipe()
write_pipe = os.fdopen(write_fd, "w")
with open(os.path.join(WORK_DIR, 'genotypes.bcf'), "w") as bcf_file:
    proc = subprocess.Popen(
        [f"{BCFTOOLS}", "view", "-O", "b"], stdin=read_fd, stdout=bcf_file
    )
    export_ts.write_vcf(write_pipe, ploidy=2, contig_id='22', individual_names=ind_labels)
    write_pipe.close()
    os.close(read_fd)
    proc.wait()
    if proc.returncode != 0:
        raise RuntimeError("bcftools failed with status:", proc.returncode)


with open(os.path.join(WORK_DIR, 'reference_inds.txt'), 'w') as OUTFILE:
    for ind in ind_labels[:nref]:
        OUTFILE.write(f'{ind}\n')

with open(os.path.join(WORK_DIR, 'target_inds.txt'), 'w') as OUTFILE:
    for ind in ind_labels[nref:]:
        OUTFILE.write(f'{ind}\n')

os.system(f"{BCFTOOLS} view --samples-file {os.path.join(WORK_DIR, 'target_inds.txt')} --output-type z --output {os.path.join(WORK_DIR, 'target_inds.vcf.gz')} {os.path.join(WORK_DIR, 'genotypes.bcf')}")
os.system(f"{BCFTOOLS} view --samples-file {os.path.join(WORK_DIR, 'reference_inds.txt')} --output-type z --output {os.path.join(WORK_DIR, 'reference_inds.vcf.gz')} {os.path.join(WORK_DIR, 'genotypes.bcf')}")


# # Make a genetic map file

os.system(f"{BCFTOOLS} query -f '%POS\\n' {os.path.join(WORK_DIR, 'genotypes.bcf')} > {os.path.join(WORK_DIR, 'site.positions')}")


positions = pd.read_csv(os.path.join(WORK_DIR, 'site.positions'), header = None)[0].values


contig.recombination_map.get_rates()


rate = contig.recombination_map.get_rates()[0] * 100 # convert to cM
cM_pos = [pos*rate for pos in positions]

with open(os.path.join(WORK_DIR, 'genetic_map.txt') , 'w') as OUTFILE:
    for i in range(len(positions)):
        OUTFILE.write(f"22\t{positions[i]}\t{cM_pos[i]:0.6f}\n")
os.system(f"head {os.path.join(WORK_DIR, 'genetic_map.txt')}")
os.system(f"tail {os.path.join(WORK_DIR, 'genetic_map.txt')}")

with open(os.path.join(WORK_DIR, 'sample_map.txt') , 'w') as OUTFILE:
    for ind_string in ind_labels[:nref]:
        pop = ind_string.split('-')[0]
        OUTFILE.write(f'{ind_string}\t{pop}\n')
os.system(f"head {os.path.join(WORK_DIR, 'sample_map.txt')}")
os.system(f"tail {os.path.join(WORK_DIR, 'sample_map.txt')}")

os.system(f"{BCFTOOLS} query -f '%POS %REF %ALT\\n' {os.path.join(WORK_DIR, 'genotypes.bcf')} > {os.path.join(WORK_DIR, 'site.alleles')}")

site_alleles = pd.read_csv(os.path.join(WORK_DIR, 'site.alleles'), comment='#', sep = ' ', header = None)
site_alleles.columns = ['pos', 'ref', 'alt']
site_REF = site_alleles['ref'].tolist()
site_ALT = site_alleles['alt'].tolist()

df = pd.DataFrame(la_mat)
#df.columns = [f'hap_{s}' for s in hap_labels]
df.insert(loc = 0, column= 'CHROM', value = 22)
df.insert(loc = 1, column= 'POS', value = positions)
df.insert(loc = 2, column= 'ID', value = [f'site_{x.id}' for x in export_ts.sites()])
df.insert(loc = 3, column= 'REF', value = site_REF)
df.insert(loc = 4, column= 'ALT', value = site_ALT)
df.insert(loc = 5, column= 'QUAL', value = '.')
df.insert(loc = 6, column= 'FILTER', value = 'PASS')
df.insert(loc = 7, column= 'INFO', value = '.')
df.insert(loc = 8, column= 'FORMAT', value = 'LA')


metadata = df.iloc[:,0:9].copy()
metadata


def vcfheader(ts, target_pop):
    from datetime import date
    fileformat = '##fileformat=VCFv4.2\n'
    fileDate = f'##filedate={date.today().ctime()}\n'
    FILTER = '##FILTER=<ID=PASS,Description="All filters passed">\n'
    contig = f'##contig=<ID=22,length={int(np.ceil(ts.get_sequence_length()))}>\n'
    FORMAT = '##FORMAT=<ID=LA,Number=1,Type=String,Description="Local ancestry">\n'

    pop_of_sample = dict(zip(range(len(ts.tables.nodes)), ts.tables.nodes.population))

    nind_in_pop = collections.defaultdict(int)
    pops = [pop_of_sample[i] for i in export_ts.samples(target_pop)]
    for p in pops:
        nind_in_pop[p] += 1
    for p in nind_in_pop:
        nind_in_pop[p] = int(nind_in_pop[p]/2)

    ind_labels = []
    for p in nind_in_pop:
        for i in range(1, nind_in_pop[p]+1):
            ind_labels.append(f'pop_{p}-ind_{i:05}')

    header = fileformat+fileDate+FILTER+contig+FORMAT
    header = header +'\t'.join(['#CHROM','POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT'])
    header = header + '\t' + '\t'.join(ind_labels) + '\n'
    return(header)

header = vcfheader(export_ts, target_pop = admixed_pop_idx)



read_fd, write_fd = os.pipe()
write_pipe = os.fdopen(write_fd, "w")
with open(os.path.join(WORK_DIR, 'la_true.bcf'), "w") as bcf_file:
    proc = subprocess.Popen(
        [f"{BCFTOOLS}", "view", "-O", "b"], stdin=read_fd, stdout=bcf_file
    )
    #print(header, file=write_pipe)
    for line in header:
        write_pipe.write(line)

    for i in range(la_mat.shape[0]):
        line = '\t'.join(metadata.iloc[i].astype(str).tolist() + list(np.char.add(np.char.add(la_mat[i, ::2].astype(str),  '|' ), la_mat[i, 1::2].astype(str)))) + '\n'
        write_pipe.write(line)

    write_pipe.close()
    os.close(read_fd)
    proc.wait()
    if proc.returncode != 0:
        raise RuntimeError("bcftools failed with status:", proc.returncode)


os.system(f"{BCFTOOLS} view --output {os.path.join(WORK_DIR, 'la_true.vcf.gz')} --output-type z {os.path.join(WORK_DIR, 'la_true.bcf')}")